The following block copies much of the code from the previous notebooks - it sets up our notebook with our data that has been "cleaned" and with its new features. See the previous parts for explanations of each step.

In [1]:
#setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

train_df = pd.read_csv('train.csv')  
test_df = pd.read_csv('test.csv')
#combine = [train_df, test_df]
test_df_predictions = pd.read_csv('test.csv')

train_df.drop('Cabin', 1, inplace=True)
test_df.drop('Cabin', 1, inplace=True)
train_df.drop('Ticket', 1, inplace=True)
test_df.drop('Ticket', 1, inplace=True)
train_df.drop('PassengerId', 1, inplace=True)
test_df.drop('PassengerId', 1, inplace=True)
train_df.drop('Embarked', 1, inplace=True)
test_df.drop('Embarked', 1, inplace=True)

train_df['Age'].fillna(train_df['Age'].median(),inplace=True)
test_df['Age'].fillna(test_df['Age'].median(),inplace=True)

test_df['Fare'].fillna(test_df['Fare'].median(),inplace=True)

df_sex = pd.get_dummies(train_df[['Sex']])
train_new = pd.concat([train_df,df_sex],axis=1)
train_new.drop('Sex',1,inplace=True)
df_sex_test = pd.get_dummies(test_df[['Sex']])
test_new = pd.concat([test_df,df_sex_test],axis=1)
test_new.drop('Sex',1,inplace=True)

def extract_title():
    train_new['Title'] = train_new['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    test_new['Title'] = test_new['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

extract_title()
train_new.drop('Name',1,inplace=True)
test_new.drop('Name',1,inplace=True)

def extractMrs():
    married_female = []
    for i in train_new['Title']:
        if i == "Mrs":
            married_female.append(1)
        else:
            married_female.append(0)
    train_new['married_female'] = married_female
    
    married_female_test = []
    for i in test_new['Title']:
        if i == "Mrs":
            married_female_test.append(1)
        else:
            married_female_test.append(0)
    test_new['married_female_test'] = married_female_test
    
    
extractMrs()
train_new.drop('Title',1,inplace=True)
test_new.drop('Title',1,inplace=True)

In [2]:
X_train = train_new.drop("Survived", axis=1)
Y_train = train_new["Survived"]
X_test = test_new
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [3]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,married_female
0,3,22.0,1,0,7.2500,0,1,0
1,1,38.0,1,0,71.2833,1,0,1
2,3,26.0,0,0,7.9250,1,0,0
3,1,35.0,1,0,53.1000,1,0,1
4,3,35.0,0,0,8.0500,0,1,0


In [4]:
logisticRegressionModel = LogisticRegression()
logisticRegressionModel.fit(X_train,Y_train)
Y_predictions = logisticRegressionModel.predict(X_test)
scoreLogisticRegressionModel = round(logisticRegressionModel.score(X_train,Y_train)*100,2)
scoreLogisticRegressionModel 
pd.DataFrame(list(zip(X_train.columns,np.transpose(logisticRegressionModel.coef_))))

,0,1
0,Pclass,[-0.938702229043]
1,Age,[-0.0391445961934]
2,SibSp,[-0.322828143331]
3,Parch,[-0.175736312102]
4,Fare,[0.0044215650881]
5,Sex_female,[2.20175254136]
6,Sex_male,[-0.188611099287]
7,married_female,[0.752735348195]


In [5]:
scoreLogisticRegressionModel

80.25

We'll also try a random forest classifier method and print out a list of the feature importances. The feature importances tells us how important each of our features was in classifying passengers.

In [6]:
randomForestModel = RandomForestClassifier(n_estimators = 100)
randomForestModel.fit(X_train,Y_train)
Y_predictions = randomForestModel.predict(X_test)
randomForestModel.score(X_train, Y_train)
#scoreRandomForestModel = round(randomForestModel.score(X_train,Y_train)*100,2)
#scoreRandomForestModel
features = X_train.columns[:8]
list(zip(X_train[features], randomForestModel.feature_importances_))

[('Pclass', 0.082517590776785082),
 ('Age', 0.25293461148099183),
 ('SibSp', 0.046237921134695263),
 ('Parch', 0.040292502385349538),
 ('Fare', 0.281185374614285),
 ('Sex_female', 0.10825188610264361),
 ('Sex_male', 0.15799748845727524),
 ('married_female', 0.030582625047974491)]

In [7]:
kaggleSubmissionFile = pd.DataFrame( { "PassengerId": test_df_predictions["PassengerId"],
                                   "Survived": Y_predictions})
kaggleSubmissionFile.to_csv('submission.csv',index=False)